### INCREMENTAL DATA LOADING 

In [0]:
CREATE DATABASE Sales

In [0]:
-- Create the table (same as your original code)
CREATE TABLE sales.Orders (
    OrderID INT,
    OrderDate DATE,
    CustomerID INT,
    CustomerName VARCHAR(100),
    CustomerEmail VARCHAR(100),
    ProductID INT,
    ProductName VARCHAR(100),
    ProductCategory VARCHAR(50),
    RegionID INT,
    RegionName VARCHAR(50),
    Country VARCHAR(50),
    Quantity INT,
    UnitPrice DECIMAL(10,2),
    TotalAmount DECIMAL(10,2)
);



In [0]:
-- Insert sample data
INSERT INTO sales.Orders VALUES
(1001, '2023-01-15', 101, 'John Smith', 'john.smith@email.com', 201, 'Wireless Headphones', 'Electronics', 1, 'North', 'USA', 2, 99.99, 199.98),
(1002, '2023-01-16', 102, 'Emily Johnson', 'emily.j@email.com', 202, 'Bluetooth Speaker', 'Electronics', 2, 'South', 'USA', 1, 79.99, 79.99),
(1003, '2023-01-17', 103, 'Michael Brown', 'michael.b@email.com', 203, 'Coffee Maker', 'Home Appliances', 3, 'East', 'Canada', 1, 129.99, 129.99),
(1004, '2023-01-18', 104, 'Sarah Davis', 'sarah.d@email.com', 204, 'Yoga Mat', 'Fitness', 4, 'West', 'USA', 3, 29.99, 89.97),
(1005, '2023-01-19', 105, 'Robert Wilson', 'robert.w@email.com', 205, 'Smart Watch', 'Electronics', 1, 'North', 'USA', 1, 199.99, 199.99),
(1006, '2023-01-20', 101, 'John Smith', 'john.smith@email.com', 206, 'Desk Lamp', 'Home', 2, 'South', 'USA', 2, 49.99, 99.98),
(1007, '2023-01-21', 106, 'Jennifer Lee', 'jennifer.l@email.com', 207, 'Backpack', 'Accessories', 3, 'East', 'Canada', 1, 59.99, 59.99),
(1008, '2023-01-22', 107, 'David Miller', 'david.m@email.com', 208, 'Water Bottle', 'Fitness', 4, 'West', 'USA', 4, 14.99, 59.96),
(1009, '2023-01-23', 108, 'Lisa Taylor', 'lisa.t@email.com', 209, 'Notebook', 'Office', 1, 'North', 'USA', 5, 9.99, 49.95),
(1010, '2023-01-24', 102, 'Emily Johnson', 'emily.j@email.com', 210, 'Phone Case', 'Accessories', 2, 'South', 'USA', 2, 19.99, 39.98);

### DATA WAREHOUSING

In [0]:
CREATE DATABASE orderDWH

## STAGING LAYER 

In [0]:
CREATE OR REPLACE TABLE orderDWH.stg_sales
as 
SELECT * FROM sales.Orders

In [0]:
SELECT * FROM orderDWH.stg_sales

### TRANSFORMATION

In [0]:
CREATE VIEW  orderDWH.v_trans_sales
AS
SELECT * FROM orderDWH.stg_sales WHERE Quantity > 1


### CORE LAYER

In [0]:
select * from orderDWH.v_trans_sales;


### **Creating customer dimension **DimCustomer****

In [0]:
CREATE or replace TABLE orderDWH.DimCustomers(
  CustomerID int,
  CustomerName STRING,
  CustomerEmail STRING,
  DimCustomerKey INT
)

In [0]:
create or replace view orderDWH.v_dim_customers
as 
select *,row_number() over (order by CustomerID) as DimCustomerKey from (select 
    Distinct(CustomerID) as CustomerID,
    CustomerName,
    CustomerEmail
FROM orderDWH.v_trans_sales) 


In [0]:
insert into table orderdwh.dimcustomers
select * from orderDWH.v_dim_customers

In [0]:
select * from orderDWH.DimCustomers

### Creating product dimension DimProduct

In [0]:
select * from orderDWH.v_trans_sales;

In [0]:
create view v_dim_products
as 
select * ,row_number() over (order by ProductID) as DimProductKey from
(SELECT 
  Distinct(ProductID) as ProductID,
  ProductName,
  ProductCategory
FROM orderdwh.v_trans_sales )


In [0]:
select * from v_dim_products

In [0]:
create or replace table orderdwh.DimProducts(
  ProductID int,
  ProductName string,
  ProductCategory string,
   DimProductKey int
)

In [0]:
insert into table orderdwh.dimproducts
select * from v_dim_products

### Creating Region Dimension DimRegion

In [0]:
create or replace view v_dim_regions
as 
select * ,row_number() over (order by RegionID) as DimRegionKey from
(SELECT 
  Distinct(RegionID) as RegionID,
  RegionName,
  Country
FROM orderdwh.v_trans_sales )


In [0]:
select * from v_dim_regions


In [0]:
create or replace table orderdwh.DimRegions(
  RegionID int,
  RegionName string,
  Country string,
   DimRegionKey int
)


In [0]:
insert into table orderdwh.dimregions
select * from v_dim_regions

###  Creating ORDER DATE dimension DimDate

In [0]:
create or replace table orderdwh.DimDate(
 OrderDate date,
 DimOrderDateKey int
)

In [0]:
create or replace view v_dim_dates
as 
select * ,row_number() over (order by OrderDate) as DimOrderDateKey from
(SELECT 
  Distinct(OrderDate) as OrderDate
FROM orderdwh.v_trans_sales )

In [0]:
insert into table orderdwh.dimdate
select * from v_dim_dates

In [0]:
select * from v_dim_dates


### FACT TABLE


In [0]:
CREATE TABLE orderdwh.FactSales
(
  OrderID INT,
  Quantity INT,
  UnitPrice INT,
  TotalAmount INT,
  DimProductKey INT,
  DimCustomerKey INT,
  DimRegionKey INT,
  DimOrderDateKey INT
  )

In [0]:
SELECT 
  F.OrderID,
  F.Quantity,
  F.UnitPrice,
  F.TotalAmount,
  C.DimCustomerKey,
  P.DimProductKey,
  R.DimRegionKey,
  D.DimOrderDateKey
From
  orderdwh.v_trans_sales F
LEFT JOIN
  orderdwh.dimCustomers C
ON
 F.CustomerID = C.CustomerID
LEFT JOIN
  orderdwh.DimProducts P
ON
 F.ProductID = P.ProductID
LEFT JOIN
  orderdwh.DimRegions R
ON
 F.RegionID = R.RegionID
LEFT JOIN
  orderdwh.DimDate D
ON
 F.OrderDate = D.OrderDate
  

In [0]:
select * from orderdwh.dimregions